# Multiple DSC sequences

This is an enhanced version of [the Quick Start tutorial](tutorials/Quick_Start.html) to demonstrate the use of multiple sequences in DSC. Material used in this document can be found in [DSC2 vignettes repo](https://github.com/stephenslab/dsc2/tree/master/vignettes/one_sample_location_winsor).


## DSC Specification and Execution
The DSC problem is similar to what we have previously worked on, i.e. comparison of location parameter estimation methods. This time we simulate data under *t* distribution (df = 2) and Cauchy distribution. Then before estimating location parameter using mean or median method, there is an **optional** `transform` step where we provide two methods for *Winsorization*. This results in two possible DSC sequences:

*  simulate -> estimate -> score
*  simulate -> transform -> estimate -> score

The DSC problem is fully specified as:

```yaml
  simulate:
      exec: rt.R, rcauchy.R
      seed: R(1:5)
      params:
          n: 1000
          true_loc: 0, 1
      return: x, true_loc

  transform:
      exec: winsor1.R, winsor2.R
      params:
          x: $x
          exec[1]:
              fraction: 0.05
          exec[2]:
              multiple: 3
      return: x

  estimate:
      exec: mean.R, median.R
      params:
          x: $x
      return: loc

  mse:
      exec: MSE.R
      params:
          mean_est: $loc
          true_mean: $true_loc
      return: mse

  DSC:
      run: simulate *
           (transform * estimate, estimate) *
           mse
      exec_path: R
      output: dsc_result
```

where Winsorization routines in `transform` block are:

```r
  ==> ../vignettes/one_sample_location_winsor/R/winsor1.R <==
  ##  replace the extreme values with limits
  winsor1 <- function (x, fraction=.05)
  {
     if(length(fraction) != 1 || fraction < 0 ||
           fraction > 0.5) {
        stop("bad value for 'fraction'")
     }
     lim <- quantile(x, probs=c(fraction, 1-fraction))
     x[ x < lim[1] ] <- lim[1]
     x[ x > lim[2] ] <- lim[2]
     return(x)
  }
  x = winsor1(x, fraction)

  ==> ../vignettes/one_sample_location_winsor/R/winsor2.R <==
  ## move the datapoints that are x times the absolute deviations from mean
  winsor2 <- function (x, multiple=3)
  {
     if(length(multiple) != 1 || multiple <= 0) {
        stop("bad value for 'multiple'")
     }
     med <- median(x)
     y <- x - med
     sc <- mad(y, center=0) * multiple
     y[ y > sc ] <- sc
     y[ y < -sc ] <- -sc
     return(y + med)
  }
  x = winsor2(x, multiple)

```

To execute the DSC on a computer using 8 CPU threads,


In [1]:
! dsc -x settings.dsc -j 8

INFO: DSC script exported to settings.html
INFO: Constructing DSC from settings.dsc ...
DSC: 100%|██████████| 31/31 [00:13<00:00,  2.91it/s]
INFO: Building output database dsc_result.rds ...
INFO: DSC complete!
INFO: Elapsed time 16.958 seconds.


*FIXME* annotation and link to shinydsc for result exploration